## Filling the database
In this notebook we fill the database with three tables, one table that holds the actual routes, and two tables which serve as helpers, listing up gyms and rock climbing holds.

So after running this our database will look like this

```
         / holds - Lists the types of rock climbing holds 
climbing - gyms  - Lists gyms, with details like location, opening hours, etc.
         \ walls - Lists the actual rock climbing walls, listing ratings, difficulties, which type of holds, the gym...
```


### Imports and presetup

In [1]:
import pyodbc

import pandas as pd 
# While pandas has a builtin read_sql and read_sql_query method, it is meant for use with SQLAlchemy and would throw warnings
# So we need to write our own quick helper func
def query_to_df(query, cursor):
    cursor.execute(query)
    cols = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    df = pd.DataFrame.from_records(data = rows, columns=cols)
    df.set_index(cols[0], inplace = True)
    return df

from dotenv import load_dotenv
import os

### Connect to the SQL database using credentials from .env, and reset the database for a clean slate

In [2]:
load_dotenv()
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

if not 'cnxn' in locals(): # type: ignore
    cnxn = pyodbc.connect("DRIVER={MySQL ODBC 9.3 Unicode Driver};"
                        "SERVER=localhost;" 
                        f"DATABASE={DB_NAME};"
                        f"USER={USER};"
                        f"PASSWORD={PASSWORD};"
                        "PORT=3306;"
                        "OPTION=3;")

cnxn.cursor().execute("DROP TABLE IF EXISTS holds;")
cnxn.cursor().execute("DROP TABLE IF EXISTS gyms;")
cnxn.cursor().execute("DROP TABLE IF EXISTS walls;")
cnxn.commit()

cursor = cnxn.cursor()

### Fill the holds table with preset hold names

In [3]:
# Create a table listing hold types by autogenerated ids
cursor.execute("""
                CREATE TABLE holds (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    name VARCHAR(255) UNIQUE NOT NULL
               );
               """)
cnxn.commit()

# Fill it with some common hold names
cursor.execute("""
                INSERT INTO holds (name)
                VALUES 
                    ('Jug'),
                    ('Edge'),
                    ('Slab'),
                    ('Crimp'),
                    ('Pinch'),
                    ('Sloper'),
                    ('Undercling')
               """)
cnxn.commit()

# Run a selection to check that we've got what we expect in the table
temp_df = query_to_df("SELECT * FROM holds ORDER BY id", cursor)
temp_df.head()


,name
id,
1,Jug
2,Edge
3,Slab
4,Crimp
5,Pinch


### Create a randomized list of gyms

#### Create a random list of gyms

In [4]:
import random

# Set up a list of states and associated cities to pull from 
states = ['Pennsylvania', 'New York', 'New Jersey']
cities = [['Philadelphia', 'Harrisburg', 'Pittsburgh'],['New York', 'Newark'],['Wilmington','Glassboro','Atlantic City']]
ZIP_Prefix = [[150,196],[10,14],[7,8]]

# Set up a list of randomized street name components
Street_init = ['Jane', 'Cherry', 'Dr Watson', 'Colonial', 'Center', 'Duck', 'Swan', 'New']
Street_terminal = [' Lane',' Street',' Boulevard',' Industrial Park', ' Drive']
Street_num = [0,350]

# Set up a list of randomized gym name components
Gym_prefix = ['Dave’s', 'Horton’s', 'Clear', 'Big', 'Climbathon']
Gym_terminal = [' Gym', ' Climb', ' Climbing Gym', ' Wall House']

# Generate every possible gym name from the above
names = [a + b for a in Gym_prefix for b in Gym_terminal]
gyms = []

# Associate each gym name with a random adress, city, state etc.
for name in names:
    state_idx = random.randint(0,len(states)-1) 
    state = states[state_idx]

    city = random.choice(cities[state_idx])

    zip_p = f"{random.randint(ZIP_Prefix[state_idx][0], ZIP_Prefix[state_idx][1]):03d}"
    zip_e = f"{random.randint(0,99):02d}"
    zip = zip_p + zip_e
    
    street = random.choice(Street_init) + random.choice(Street_terminal) + ' ' + str(random.randint(Street_num[0], Street_num[1]))
    
    gyms.append([name,state,city,zip,street])

random.shuffle(gyms) # Shuffle the gym list to keep all the Dave's from clumping 

#### Create a table and fill it with the randomized gyms

In [5]:
# Create a table listing gyms that can hold all the data we wanted to add
cursor.execute("""
                CREATE TABLE gyms (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    Name VARCHAR(255) UNIQUE NOT NULL,
                    State VARCHAR(255) NOT NULL,
                    City VARCHAR(255) NOT NULL,
                    ZipCode VARCHAR(255) NOT NULL,
                    Address VARCHAR(255) NOT NULL
               );
               """)
cnxn.commit()

# Put together a dynamic insertion string that 
insert_root = """
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
"""

insert = ''

for gym in gyms:
    insertion_str = ''
    for ele in gym:
        insertion_str = insertion_str + '\'' + ele + '\'' + ','

    insert = insert + '    (' + insertion_str[0:-1] + '),' + '\n'

insert = insert_root + insert[0:-2]

# Let's show what the somewhat messily constructed insertion string looks like
print('Insertion: ', end='\n---------------------')
print(insert)

# And Run it
cnxn.execute(insert)
cnxn.commit()

# Then just like before let's check that the insertion did what we expected
print('\n\nResult')
print('---------------------')
temp_df = query_to_df("SELECT * FROM gyms ORDER BY id", cursor)
temp_df.head()

Insertion: 
---------------------
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
    ('Clear Climb','New York','New York','01396','Swan Boulevard 292'),
    ('Horton’s Climbing Gym','New Jersey','Atlantic City','00790','Colonial Lane 34'),
    ('Big Gym','New York','New York','01258','New Drive 267'),
    ('Climbathon Climbing Gym','New York','New York','01424','Swan Industrial Park 241'),
    ('Clear Gym','New York','New York','01177','Duck Industrial Park 74'),
    ('Dave’s Wall House','New York','Newark','01486','Jane Drive 165'),
    ('Dave’s Climb','New York','Newark','01112','Swan Industrial Park 0'),
    ('Horton’s Gym','New Jersey','Wilmington','00859','Center Drive 331'),
    ('Clear Climbing Gym','Pennsylvania','Pittsburgh','16315','Colonial Lane 289'),
    ('Big Climbing Gym','New Jersey','Glassboro','00787','Dr Watson Drive 1'),
    ('Big Wall House','New Jersey','Glassboro','00736','Dr Watson Lane 14'),
    ('Horton’s Wall House','Pennsylvania','Pittsbu

,Name,State,City,ZipCode,Address
id,,,,,
1,Clear Climb,New York,New York,01396,Swan Boulevard 292
2,Horton’s Climbing Gym,New Jersey,Atlantic City,00790,Colonial Lane 34
3,Big Gym,New York,New York,01258,New Drive 267
4,Climbathon Climbing Gym,New York,New York,01424,Swan Industrial Park 241
5,Clear Gym,New York,New York,01177,Duck Industrial Park 74


### Now we can finally make some climbing-walls and associate them with gyms

In [ ]:
# Set up a new table that stores gyms and links them to their respective gyms.
cursor.execute("""
                CREATE TABLE walls (
                    id SMALLINT PRIMARY KEY AUTO_INCREMENT , 
                    gym SMALLINT,
                    FOREIGN KEY (gym) REFERENCES gyms(id)
               );
               """)
cnxn.commit()

